In [1]:
import numpy as np
import pandas as pd
import reverse_geocoder as rg
from sklearn.cluster import MeanShift
import pycountry

## 1

Загружаем данные о посещениях заведений: 
- checkins.dat: отмечает чекины (посещения) пользователей на площадках. Каждая регистрация имеет уникальный             идентификатор,              а       также идентификатор пользователя и идентификатор места проведения.
- Очищаем данные от записей с пропусками. 
- Выводим количество записей после очистки

In [2]:
with open('../datasets/checkins.dat', 'r') as f:
    next(f)  # пропустить первую строку
    df = pd.DataFrame([l.rstrip().split('|') for l in f],
                      columns=['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'creat_at'])

# Вывод DataFrame
df = df.drop(0)
df = df[['latitude', 'longitude', 'venue_id', 'id', 'user_id']].apply(
    pd.to_numeric, errors='coerce')
df = df.dropna()
len(df)

396634

С помощью геолокаций и библиотеки [Reverse Geocoder](https://github.com/thampiman/reverse-geocoder),
узнай страну для каждой геопозиции.
Выводим **название** второй страны по количеству записей.

In [3]:
# Создание кортежей из координат для использования в Reverse Geocoder
coordinates = list(zip(df['latitude'], df['longitude']))

# Определение страны для каждой геопозиции
results = rg.search(coordinates)

# Создание столбца 'cc' в DataFrame 'df'
df['cc'] = [result['cc'] for result in results]

# Подсчет количества записей для каждой страны
country_counts = df['cc'].value_counts()

# используется библиотека pycountry для получения имени страны на основе ее кода (alpha_2),
# который извлекается из индекса 'country_counts' второго элемента.
country_2 = pycountry.countries.get(alpha_2=country_counts.index[1]).name
country_2

Loading formatted geocoded file...


'Indonesia'

## 3

Так как нас интересуют только Американские локации:
- Очищаем данные от локации других стран
- Оставляем 50 самых часто встречаемых заведений (venue_id).
- Выводим количество локаций, оставшихся после этих очисток.

In [4]:
# Фильтрация данных только для американских локаций
american_df = df[df['cc'] == 'US']

# Подсчет количества записей для каждого заведения (venue_id)
venue_counts = american_df['venue_id'].value_counts()

# Оставление только 50 самых часто встречаемых заведений
top_50_venues = venue_counts.head(50).index

# Очистка данных от локаций, не входящих в топ 50 заведений
cleaned_df = american_df[american_df['venue_id'].isin(top_50_venues)]

# Вывод количества локаций после очисток
len(cleaned_df)

162099

## 4

Решаем задачу кластеризации. 
- Пользуемся алгоритмом [Mean Shift] для кластеризации локаций. 
- В параметрах указываем `MeanShift(bandwidth=0.1, bin_seeding=True)`:
    - `bandwidth=0.1` - это ширина ядра кластеризации. Для средних широт США - это порядка 5-10 км. 
    - `bin_seeding=True` - для ускорения работы алгоритма.
    
Выводим количество кластеров.

In [5]:
# Выбор столбцов latitude и longitude для кластеризации
coordinates = cleaned_df[['latitude', 'longitude']]

ms = MeanShift(bandwidth=0.1, bin_seeding=True)
ms.fit(coordinates)

# После обучения модели MeanShift получаются координаты центров кластеров с помощью атрибута cluster_centers_.
# Эти центры кластеров представляют оцененные местоположения кластеров в данных.
cluster_centers = ms.cluster_centers_

# Количество кластеров определяется путем нахождения уникальных меток (labels),
# которые были присвоены каждой точке данных в атрибуте ms.labels_.
# Функция np.unique() используется для получения уникальных меток.
num_clusters = len(np.unique(ms.labels_))
num_clusters

2846

## 5

Находим те центры кластеров, которые наиболее близко находятся к офисам продаж компании.\
Загружаем [данные по координатам офисов компании](datasets/offices.csv). Для каждого офиса найди 5 самых ближайших к нему центров кластеров. \
(Пренебрежем тем, что Земля круглая и рассчитаем Евклидово расстояние).\
У компании 11 офисов, значит у нас должно получится 55 мест установки баннеров. \
Выведем координаты установки баннера, который ближе всего находится к офису компании.

In [6]:

offices = pd.read_csv('../datasets/offices.csv')

# Извлечение координат офисов из датасета
office_coordinates = offices[['latitude', 'longitude']]

# Вычисляем расстояния между офисами и центрами кластеров на основе их координат (широты и долготы) 
 
# 1 office_coordinates['latitude'].values[:, np.newaxis] - с помощью np.newaxis 
#   добавляется новое измерение в массив широты, чтобы его форма стала (n, 1), где n - количество офисов.
# 2 cluster_centers[:, 0] - это массив значений широты центров кластеров.
# 3 office_coordinates['latitude'].values[:, np.newaxis] - cluster_centers[:, 0] - вычитание 
#   поэлементно выполняется между массивами широты офисов и центров кластеров.
# 4 office_coordinates['longitude'].values[:, np.newaxis] - cluster_centers[:, 1] - выполняется 
#   вычитание между массивами долготы офисов и центров кластеров.

distances = np.sqrt((office_coordinates['latitude'].values[:, np.newaxis] - cluster_centers[:, 0]) ** 2 +
                    (office_coordinates['longitude'].values[:, np.newaxis] - cluster_centers[:, 1]) ** 2)

#   Нахождение 5 самых ближайших центров кластеров для каждого офиса
# 1 Функция argsort() из библиотеки  используется для получения индексов элементов массива distances, 
#   отсортированных в порядке возрастания. Здесь axis=1 указывает, что сортировка производится по строкам, 
#   то есть для каждого офиса сортируются расстояния до центров кластеров.
closest_clusters = np.argsort(distances, axis=1)[:, :5]

# Создание списка словарей с информацией о ближайших центрах кластеров для каждого офиса
banner_locations_data = []

# Этот цикл  перебирает индекс i и кортеж (office_index, _) для каждой строки в датафрейме offices.
# Функция enumerate() добавляет счетчик i к каждой строке, чтобы можно было отслеживать текущий индекс строки.
# offices.iterrows() возвращает итератор, который позволяет перебирать строки (индекс и значения) в датафрейме offices. 
for i, (office_index, _) in enumerate(offices.iterrows()):
    office_name = int(office_index)
    # Из массива cluster_centers выбираются координаты ближайших центров кластеров 
    # для текущего офиса. closest_clusters[i] представляет индексы этих центров.
    cluster_coords = cluster_centers[closest_clusters[i]]
    
    # Расчет Расстояния Между Офисом И Каждым Ближайшим Центром Кластера
    # office_coordinates['latitude'].values[i] представляет широту конкретного офиса с индексом i.
    # cluster_coords[:, 0] представляет массив широт всех ближайших центров кластеров для данного офиса.
    # (office_coordinates['latitude'].values[i] - cluster_coords[:, 0]) вычисляет разницу между широтой данного офиса 
    # и широтами ближайших центров кластеров.
    # Аналогично, office_coordinates['longitude'].values[i] - cluster_coords[:,1])
    
    distances_to_clusters = np.sqrt((office_coordinates['latitude'].values[i] - cluster_coords[:, 0]) ** 2 +
                                    (office_coordinates['longitude'].values[i] - cluster_coords[:, 1]) ** 2)
    
    # Цикл перебирает каждое расстояние и добавляет его в список словарей
    for j, distance in enumerate(distances_to_clusters):
        banner_locations_data.append({'index_office': office_name,
                                      'Cluster_Center_Latitude': cluster_coords[j, 0],
                                      'Cluster_Center_Longitude': cluster_coords[j, 1],
                                      'Distance': distance})

# Создание нового датасета banner_locations 
banner_locations = pd.DataFrame(banner_locations_data)

# Преобразование столбца 'index_office' в категориальный тип данных
banner_locations['index_office'] = banner_locations['index_office'].astype('category')

min_distance_index = banner_locations['Distance'].idxmin()

# Извлечь строку с минимальным расстоянием
closest_banner = banner_locations.loc[[min_distance_index]]

closest_banner 

,index_office,Cluster_Center_Latitude,Cluster_Center_Longitude,Distance
45,9,32.785318,-79.924742,0.004092


## 6

Отображаем точки расположения баннеров

In [7]:
import plotly.express as px

banner_locations = banner_locations.sort_values(by='Distance')
fig = px.scatter_mapbox(banner_locations,
                        lat="Cluster_Center_Latitude",
                        lon="Cluster_Center_Longitude",
                        color="index_office",
                        zoom=2.5,
                        mapbox_style='open-street-map'
                        )


# Отображение графика
fig.show()